In [ ]:
!pip install dgl
!pip install sklearn

In [ ]:
import dgl
import dgl.function as fn
import torch as th
import torch.nn as nn
import torch.nn.functional as F
from dgl import DGLGraph


gcn_msg = fn.copy_src(src='h', out='m')
gcn_reduce = fn.sum(msg='m', out='h')

In [ ]:
#from sklearn.datasets import fetch_mldata
#from sklearn.model_selection import train_test_split
#from sklearn.ensemble import RandomForestClassifier
#from sklearn.multiclass import OneVsRestClassifier
from sklearn.metrics import precision_recall_curve, roc_curve
#from sklearn.preprocessing import label_binarize

import matplotlib.pyplot as plt

In [ ]:
#message passing function
#formatting this one 
import math
import torch
import numpy 
def find_common_in_nbrs(g, i, j):
    """Find nodes that have edges to both node i and node j.
    
    Returns
    --------
    NumPy array of int64
    """
    return len(numpy.intersect1d(g.in_edges(i)[0], g.in_edges(j)[0])),len(numpy.union1d(g.in_edges(i)[0], g.in_edges(j)[0]))


def my_msg(edges):
    x = edges.src['h']
    (a,b)=x.size()
    src, dst, _ = edges.edges()
    source_nd = src.detach().numpy()
    destin_nd = src.detach().numpy()
    #print(src)
    my_wei_product=np.zeros(a)
    
    yo_2 = g.out_degrees(source_nd).detach().numpy()    
    yo_1 = g.out_degrees(destin_nd).detach().numpy()
    summation =0
     
    for i in range(a):
        inter, union = find_common_in_nbrs(g, source_nd[i], destin_nd[i])
        my_wei_product[i]= (inter/union)#math.sqrt((yo_2[i])*(yo_1[i]))
       # summation = summation + my_wei_product[i]
       # if (my_wei_product[i]<0.25):
       #    my_wei_product[i] = 0
       # if (my_wei_product[i]<0.5):
       #    my_wei_product[i] = 0.5  
       # if (my_wei_product[i]<0.75):
       #    my_wei_product[i] = 1 
       # else:
       #    my_wei_product[i] = 2 
       #     '''
    #for i in range(a):
        x[i,:] = x[i,:] * (math.exp(my_wei_product[i]))#\/summation) 

    return {'m': x}

In [ ]:
#aggregator function
def my_reduce(nodes):#aggregator function 
    messages_neg = nodes.mailbox['m']
    x = nodes.mailbox['m'].detach().numpy()
    a,b,c = messages_neg.size()#(N,D,feature_size)
    y = nodes.data['h'].detach().numpy() #(N,feature_size)
    my_dot_product=np.zeros((a,b))
    for i in range(a):
        for j in range(b):
                    #my_dot_product[i,j] = 2+(np.dot(x[i,j,:],y[i,:])/ (np.linalg.norm(x[i,j,:])* np.linalg.norm(y[i,:])))
            my_dot_product[i,j] = ((1+(np.dot(x[i,j,:],y[i,:])/ (np.linalg.norm(x[i,j,:])*np.linalg.norm(y[i,:])))))
            messages_neg[i,j,:] = messages_neg[i,j,:]*my_dot_product[i,j]
    agg_=torch.sum(messages_neg, dim=1)
    #agg_=torch.mean(messages_neg, dim=1)
    #agg_=messages_neg.sum(1)
    return {'h': agg_}

In [ ]:
class GCNLayer(nn.Module):
    def __init__(self, in_feats, out_feats):
        super(GCNLayer, self).__init__()
        self.linear = nn.Linear(in_feats, out_feats)

    def forward(self, g, feature):
        # Creating a local scope so that all the stored ndata and edata
        # (such as the `'h'` ndata below) are automatically popped out
        # when the scope exits.
        with g.local_scope():
            g.ndata['h'] = feature
            g.update_all(my_msg,  my_reduce)
            #g.update_all(gcn_msg, gcn_reduce)
            h = g.ndata['h']
            return self.linear(h)

In [ ]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.layer1 = GCNLayer(1433, 16)#1433 for cora, 500, 1024,3703 for citseer  
        self.layer2 = GCNLayer(16, 7)

    def forward(self, g, features):
        x = F.relu(self.layer1(g, features))
        x = self.layer2(g, x)
        #print("The proba vector is",x,'and its size is',x.shape)
        return x
net = Net()
print(net)

In [ ]:
from dgl import DGLGraph
from dgl.data import citation_graph as citegrh
import networkx as nx
from dgl.data import CiteseerGraphDataset,CoraGraphDataset
import numpy as np
def load_cora_data():
    dataset = CoraGraphDataset()
    g = dataset[0]
    features = g.ndata['feat']
    labels = g.ndata['label']
    train_mask = np.add(g.ndata['val_mask'], g.ndata['train_mask'])
    test_mask = g.ndata['test_mask']
    #print(train_mask.shape())
    n = g.ndata['val_mask'].size()
    n=n[0]
    print(n)
    for i in range(n):
        train_mask[i] = False
        test_mask[i] = False
    
    train = (int)(n*0.3)
    
    for i in range(train):
        train_mask[i] = True
    
    for i in range(train,n-(int)(n*0.1)):
        test_mask[i] = True


    return g, features, labels, train_mask, test_mask

In [ ]:
'''
from dgl.data import CoraGraphDataset
def load_cora_data():
    dataset = CoraGraphDataset()
    g = dataset[0]
    features = g.ndata['feat']
    labels = g.ndata['label']
    train_mask = g.ndata['train_mask']
    test_mask = g.ndata['test_mask']
    n = g.ndata['train_mask'].size()
    n=n[0]
    print(n)
    
    for i in range(n):
        train_mask[i] = False
        test_mask[i] = False
    
    train=810
    
    for i in range(train):
        train_mask[i] = True
    
    for i in range(train+1,n):
        test_mask[i] = True

    return g, features, labels, train_mask, test_mask

from dgl.data import PubmedGraphDataset
def load_pubmed_data():
    dataset = PubmedGraphDataset()
    g = dataset[0]
    features = g.ndata['feat']
    labels = g.ndata['label']
    train_mask = np.add(g.ndata['val_mask'], g.ndata['train_mask'])
    test_mask = g.ndata['test_mask']
    #print(train_mask.shape())
    n = g.ndata['val_mask'].size()
    n=n[0]
    print(n)
    for i in range(n):
        train_mask[i] = False
        test_mask[i] = False
    
    train = (int)(n*0.3)
    
    for i in range(train):
        train_mask[i] = True
    
    for i in range(train,n-331):
        test_mask[i] = True


    return g, features, labels, train_mask, test_mask
'''

In [ ]:
def evaluate(model, g, features, labels, mask):
    model.eval()
    with th.no_grad():
        logits = model(g, features)
        logits = logits[mask]
        labels = labels[mask]
        _, indices = th.max(logits, dim=1)
        correct = th.sum(indices == labels)
        return correct.item() * 1.0 / len(labels)

In [ ]:
import time
import numpy as np
g, features, labels, train_mask, test_mask = load_cora_data()
print(labels)

In [ ]:
import time
import numpy as np
g, features, labels, train_mask, test_mask = load_cora_data()
# Add edges between each node and itself to preserve old node representations
prob_y_train=[]
g.add_edges(g.nodes(), g.nodes())
optimizer = th.optim.Adam(net.parameters(), lr=1e-2)
dur = []
GNN_layer=[]
max_accuracy=0
log=[]
for epoch in range(200):
    if epoch >=3:
        t0 = time.time()

    net.train()
    logits= net(g, features)
    log.append(logits)
    
    logp = F.log_softmax(logits, 1)
    prob_y_train.append(logp)
    loss = F.nll_loss(logp[train_mask], labels[train_mask])

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if epoch >=3:
        dur.append(time.time() - t0)

    acc = evaluate(net, g, features, labels, test_mask)
    print("Epoch {:05d} | Loss {:.4f} | Test Acc {:.4f} | Time(s) {:.4f}".format(
            epoch, loss.item(), acc, np.mean(dur)))

In [ ]:
our_confidence=np.exp(prob_y_train[16].detach().numpy())
print(type(our_confidence))
print(len(our_confidence),'x',len(our_confidence[0]))

<class 'numpy.ndarray'>
2708 x 7


In [ ]:
print(our_confidence)
print(labels)

In [ ]:
labels=list(labels)
okay=[]
for i in our_confidence:
    okay.append(list(i))

In [ ]:
import pandas as pd
result_df = pd.DataFrame(list(zip(okay)),
               columns =['confidence'])
result_df.to_csv('/content/drive/MyDrive/Confidence/GCN.csv',sep=',', encoding='utf-8')

In [ ]:
#writing into CSV file
import csv
data=[]


header = ['0','1','2','3','4','5','6']
#labels
for i in labels:#our_confidence:
    yu_mama=[0,0,0,0,0,0,0]
    yu_mama[i]=1
    data.append(yu_mama)
 

with open('/content/drive/MyDrive/cora_dataset_label.csv', 'w', encoding='UTF8', newline='') as f:
    writer = csv.writer(f)

    # write the header
    writer.writerow(header)

    # write multiple rows
    writer.writerows(data)

In [ ]:
#not run
'''
!pip install xlswriter
import xlswriter as xv
 
# Workbook() takes one, non-optional, argument
# which is the filename that we want to create.
workbook = xv.Workbook('hello.xlsx')
 
# The workbook object is then used to add new
# worksheet via the add_worksheet() method.
worksheet = workbook.add_worksheet()
 
# Use the worksheet object to write
# data via the write() method.
for i in our_confidence:
    worksheet.write(i)

#worksheet.write('A1', 'Hello..')
#worksheet.write('B1', 'Geeks')
#worksheet.write('C1', 'For')
#worksheet.write('D1', 'Geeks')
 
# Finally, close the Excel file
# via the close() method.
workbook.close()
'''

In [ ]:
from sklearn.manifold import TSNE as tsne 
X=np.array(features)
X_embedded = tsne(n_components=2).fit_transform(X)

In [ ]:
color=[(0,0,0), (0,0,1), (0,1,0), (1,0,0), (0,1,1), (1,0,1), (1,1,0)]
colors=[]
size=[]
for i in labels:
    colors.append(color[i+2])
    size.append(1)
sizes=np.asarray(size)

In [ ]:
plt.figure(figsize=(7,7))
plt.scatter(x=X_embedded[:,0],y=X_embedded[:,1],s=sizes*10,c=colors)
plt.plot()

In [ ]:
from sklearn.manifold import TSNE as tsne 
Xr=np.array(log)
Xr=np.transpose(Xr)
Xr= Xr
print(type(Xr))
print(Xr)
X_embedded = tsne(n_components=2).fit_transform(Xr[7].detach().numpy())

In [ ]:
plt.figure(figsize=(7,7))
plt.scatter(x=X_embedded[:,0],y=X_embedded[:,1],s=sizes*10,c=colors)
plt.plot()

In [ ]:
lbs=np.zeros((2708,7))
for i in range(labels.shape[0]):
    lbs[i][labels[i]]=1
    

In [ ]:
#precision recall curve
precision = dict()
recall = dict()
for epoch in range(50):
    for i in range(7):
        yo_=prob_y_train[epoch][:][i].detach().numpy()
        yo_=(yo_-np.min(yo_))/np.ptp(yo_)
        #yo_=(yo_-np.mean(yo_))/np.std(yo_)
        print(yo_)
        precision[i], recall[i], _ = precision_recall_curve(lbs[:][i],yo_)#prob_y_train[epoch][:][i].detach().numpy()
        #print("The pr and re are :", precision[i], recall[i])
        plt.plot(recall[i], precision[i], lw=2, label='class {}'.format(i))
    
plt.xlabel("recall")
plt.ylabel("precision")

plt.title("precision vs. recall curve")
plt.show()

In [ ]:
# roc curve
fpr = dict()
tpr = dict()

for i in range(50):
    yo_=prob_y_train[epoch][:][i].detach().numpy()
    yo_=(yo_-np.min(yo_))/np.ptp(yo_)
    #yo_=(yo_-np.mean(yo_))/np.std(yo_)
    #print(yo_)
    fpr[i], tpr[i], _ = roc_curve(lbs[:][i],yo_)
    plt.plot(fpr[i], tpr[i], lw=2, label='class {}'.format(i))

plt.xlabel("false positive rate")
plt.ylabel("true positive rate")
#plt.legend(loc="best")
plt.title("ROC curve")
plt.show()